In [ ]:
# full code:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# =============================
# 1. Load Data
# =============================
df = pd.read_csv("midlevel.csv")

# =============================
# 2. Data Cleaning
# =============================

# Fill missing names with "Unknown"
df["full_name"] = df["full_name"].fillna("Unknown")

# Fill missing salaries with median
df["monthly_salary"] = df["monthly_salary"].fillna(df["monthly_salary"].median())

# Outlier removal using IQR
Q1 = df["monthly_salary"].quantile(0.25)
Q3 = df["monthly_salary"].quantile(0.75)
IQR = Q3 - Q1
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR
df = df[(df["monthly_salary"] >= lower_bound) & (df["monthly_salary"] <= upper_bound)]

# Standardize city names (fix capitalization + merge aliases)
df["city"] = df["city"].str.strip().str.title()
df["city"] = df["city"].replace({"Alex": "Alexandria", "Alexandria ": "Alexandria"})

# Round salaries
df["monthly_salary"] = df["monthly_salary"].round(2)

# =============================
# 3. Feature Engineering
# =============================

# Salary band
conditions = [
    (df["monthly_salary"] < 5000),
    (df["monthly_salary"].between(5000, 7000, inclusive="both")),
    (df["monthly_salary"] > 7000),
]
values = ["Low", "Medium", "High"]
df["salary_band"] = np.select(conditions, values, default="Unknown")

# Date features
df["joining_date"] = pd.to_datetime(df["joining_date"])
df["year"] = df["joining_date"].dt.year
df["month"] = df["joining_date"].dt.month
df["day_of_week"] = df["joining_date"].dt.day_name()
df["is_weekend"] = df["day_of_week"].isin(["Saturday", "Sunday"])

# Experience level
df["age"] = df["age"].fillna(df["age"].mean())
conditions = [
    (df["age"] < 25),
    (df["age"].between(25, 35, inclusive="both")),
    (df["age"] > 35),
]
values = ["Junior", "Mid", "Senior"]
df["experience_level"] = np.select(conditions, values, default="Unknown")

# =============================
# 4. Grouping & Aggregation
# =============================

# Average salary per city per year
avg_sal = df.groupby(["city", "year"])["monthly_salary"].mean().round(2).reset_index()

# Top earning city each year
top_cities = avg_sal.loc[avg_sal.groupby("year")["monthly_salary"].idxmax()]

# Most common name per city
most_common_name = (
    df.groupby("city")["full_name"]
    .agg(lambda x: x.mode().iloc[0] if not x.mode().empty else "Unknown")
    .reset_index()
)

# Employee counts per salary_band per city
salary_band_counts = (
    df.groupby(["city", "salary_band"])["employee_id"]
    .count()
    .reset_index(name="employee_count")
)

# =============================
# 5. Pivot Tables
# =============================

# Pivot: Average salary by year/city
pivot_avg_sal = df.pivot_table(
    index="year", columns="city", values="monthly_salary", aggfunc="mean"
).round(2)

# Pivot: Employee count by city/salary_band
pivot_count = df.pivot_table(
    index="city", columns="salary_band", values="employee_id", aggfunc="count", fill_value=0
)

# =============================
# 6. Visualization
# =============================

# Salary trend over time per city
plt.figure(figsize=(10, 6))
for city in df["city"].unique():
    city_data = df[df["city"] == city].groupby("year")["monthly_salary"].mean().reset_index()
    plt.plot(city_data["year"], city_data["monthly_salary"], marker="o", label=city)
plt.title("Average Salary Trend Over Time per City")
plt.xlabel("Year")
plt.ylabel("Average Monthly Salary")
plt.legend()
plt.show()

# Distribution of salary bands by city (stacked bar chart)
pivot_count.plot(kind="bar", stacked=True, figsize=(10, 6))
plt.title("Distribution of Salary Bands by City")
plt.xlabel("City")
plt.ylabel("Employee Count")
plt.show()

# Age vs Salary scatterplot, color-coded by city
plt.figure(figsize=(10, 6))
for city in df["city"].unique():
    subset = df[df["city"] == city]
    plt.scatter(subset["age"], subset["monthly_salary"], label=city, alpha=0.6)
plt.title("Age vs Salary by City")
plt.xlabel("Age")
plt.ylabel("Monthly Salary")
plt.legend()
plt.show()


In [ ]:
import pandas as pd
import numpy as np

df = pd.read_csv('midlevel.csv')
# print(df.head())
# print(df.info())
# print(df.describe())
# print(df.isnull().sum())

'''Detect and remove outliers in salary (e.g., anything outside 1.5×IQR).

Standardize city names (if “cairo” vs “Cairo” exists, fix capitalization).

Create a new column salary_band:

<5000 = Low,

5000–7000 = Medium,

>7000 = High.'''

df["full_name"] = df['full_name'].fillna('Unknown')
# Detect and remove outliers in salary (e.g., anything outside 1.5×IQR).
df['monthly_salary'].fillna(df['monthly_salary'].median(), inplace=False)

# Standardize city names (if “cairo” vs “Cairo” exists, fix capitalization).
# merge alex and alexandria and fix capitalization
df['city'] = df['city'].replace(['alex', 'alexandria'], 'alexandria')
df['city']=df['city'].str.title()

# Create a new column salary_band: <5000 = Low, 5000–7000 = Medium, >7000 = High.
df['monthly_salary']= df['monthly_salary'].round(2)
df['monthly_salary']= df['monthly_salary'].fillna(df['monthly_salary'].mean())
conditions =[
    (df['monthly_salary'] < 5000),
    (df['monthly_salary'] >= 5000) & (df['monthly_salary'] <= 7000),
    (df['monthly_salary'] > 7000)
]
values = ['Low', 'Medium', 'High'] 
df['salary_band'] = np.select(conditions, values,default='Unknown')
df['monthly_salary']= df['monthly_salary'].round(2)
# print(df['salary_band'].value_counts())

df

In [ ]:
'''2. Feature Engineering

Extract month and day of week from date.

Create a new column is_weekend (Saturday/Sunday = True).

Create an experience_level column:

age < 25 → Junior,

25–35 → Mid,

>35 → Senior.'''
# Extract month and day of week from date.
df['joining_date']=pd.to_datetime(df['joining_date'])
df['month']=df['joining_date'].dt.month
df['day_of_week']=df['joining_date'].dt.day_name()

# Create a new column is_weekend (Saturday/Sunday = True).
df['is_weekend'] = df['day_of_week'].isin(['Saturday', 'Sunday'])


# Create an experience_level column: age < 25 → Junior, 25–35 → Mid, >35 → Senior.
df['age']=df['age'].fillna(df['age'].mean())
conditions = [
    (df['age'] < 25),
    (df['age'] >= 25) & (df['age'] <= 35),
    (df['age'] > 35)
]
values = ['Junior', 'Mid', 'Senior']
df['experience_level']=np.select(conditions,values,default='Unknown')
# print(df['experience_level'].value_counts())
# print(df.isnull().sum())

df

In [ ]:
'''Grouping & Aggregation

Find average salary per city per year.

Find top earning city each year.

Find most common name per city.

Count how many employees fall into each salary_band per city.'''
df['year']=df['joining_date'].dt.year

# Find average salary per city per year.
avg_sal = df.groupby(['city','year'])['monthly_salary'].mean().round(2).reset_index()
print(avg_sal)

# Find top earning city each year.
top_cities = avg_sal.loc[avg_sal.groupby('year')['monthly_salary'].idxmax()]

print(top_cities)

# Find most common name per city.
most_common_name = df.groupby('city')['full_name'].agg(lambda x: x.mode().iat[0]).reset_index()
print(most_common_name)

# Count how many employees fall into each salary_band per city
print(df.groupby(['city', 'salary_band'])['employee_id'].count().reset_index(name='employee_count'))

In [ ]:
"""4. Pivot Tables & Reshaping

Make a pivot table: rows = year, columns = city, values = average salary.

Make another pivot: rows = city, columns = salary_band, values = count of employees."""

# Make a pivot table: rows = year, columns = city, values = average salary.
pivot_avg_sal = df.pivot_table(index='year', columns='city', values='monthly_salary', aggfunc='mean').round(2)
print(pivot_avg_sal)


#Make another pivot: rows = city, columns = salary_band, values = count of employees.
pivot_count = df.pivot_table(index='city', columns='salary_band', values='employee_id', aggfunc='count', fill_value=0)
print(pivot_count)


In [ ]:
"""Plot average salary trend over time per city (lineplot).

Plot distribution of salary bands by city (stacked bar chart).

Plot age vs salary scatterplot, color-coded by city."""
import matplotlib.pyplot as plt
# Plot average salary trend over time per city (lineplot).
plt.figure(figsize=(10,6))
for city in df['city'].unique():
    city_data = df[df['city'] == city].groupby('year')['monthly_salary'].mean().reset_index()
    plt.plot(city_data['year'], city_data['monthly_salary'], marker='o', label=city)
plt.title('Average Salary Trend Over Time per City')
plt.xlabel('Year')
plt.ylabel('Average Monthly Salary')
plt.legend()